In [5]:
import numpy as np
import pandas as pd
from plotnine import *

In [2]:
import pandas as pd
from kats.models.prophet import ProphetModel, ProphetParams
from kats.consts import TimeSeriesData
from kats.models.var import VARModel, VARParams
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error
import math
import numpy as np
import matplotlib.pyplot as plt

ModuleNotFoundError: No module named 'kats'

In [6]:
# Cargar dataset 
df = pd.read_csv('../Data/1900_2021_DISASTERS.xlsx - emdat data.csv')

In [7]:
# Mostrar df
df.head()

,Year,Seq,Glide,Disaster Group,Disaster Subgroup,Disaster Type,Disaster Subtype,Disaster Subsubtype,Event Name,Country,...,No Affected,No Homeless,Total Affected,Insured Damages ('000 US$),Total Damages ('000 US$),CPI,Adm Level,Admin1 Code,Admin2 Code,Geo Locations
0,1900,9002,NaN,Natural,Climatological,Drought,Drought,NaN,NaN,Cabo Verde,...,NaN,NaN,NaN,NaN,NaN,3.221647,NaN,NaN,NaN,NaN
1,1900,9001,NaN,Natural,Climatological,Drought,Drought,NaN,NaN,India,...,NaN,NaN,NaN,NaN,NaN,3.221647,NaN,NaN,NaN,NaN
2,1902,12,NaN,Natural,Geophysical,Earthquake,Ground movement,NaN,NaN,Guatemala,...,NaN,NaN,NaN,NaN,25000.0,3.350513,NaN,NaN,NaN,NaN
3,1902,3,NaN,Natural,Geophysical,Volcanic activity,Ash fall,NaN,Santa Maria,Guatemala,...,NaN,NaN,NaN,NaN,NaN,3.350513,NaN,NaN,NaN,NaN
4,1902,10,NaN,Natural,Geophysical,Volcanic activity,Ash fall,NaN,Santa Maria,Guatemala,...,NaN,NaN,NaN,NaN,NaN,3.350513,NaN,NaN,NaN,NaN


In [8]:
# Extraer columnas del df Year, Month, Country, Location, Precipitation (MM), Temperature, Disaster Type, Disaster Probability (or prediction)

df = df[['Year', 'Country', 'Location', 'Start Month', 'Disaster Type']]
df.head()

# Cambiar nombre de la columna 'Start Month' a 'Month'
df.rename(columns={'Start Month': 'Month'}, inplace=True)

In [9]:
# Añadir a la tabla Precipatation y Temperature
df['Precipitation (MM)'] = 0
df['Temperature'] = 0
df.head()

# Rellenar Precipitation (MM) con valores random entre 0 y 100
df['Precipitation (MM)'] = np.random.randint(0, 100, df.shape[0])

# Rellenar Temperature con valores random entre -10 y 40
df['Temperature'] = np.random.randint(-10, 40, df.shape[0])
df.head() 


,Year,Country,Location,Month,Disaster Type,Precipitation (MM),Temperature
0,1900,Cabo Verde,Countrywide,NaN,Drought,17,21
1,1900,India,Bengal,NaN,Drought,94,35
2,1902,Guatemala,"Quezaltenango, San Marcos",4.0,Earthquake,32,15
3,1902,Guatemala,NaN,4.0,Volcanic activity,15,39
4,1902,Guatemala,NaN,10.0,Volcanic activity,8,37


In [10]:
# Eliminar las filas con 'Mouth' o 'Day' Null
df = df.dropna(subset=['Month'])
df.head()

,Year,Country,Location,Month,Disaster Type,Precipitation (MM),Temperature
2,1902,Guatemala,"Quezaltenango, San Marcos",4.0,Earthquake,32,15
3,1902,Guatemala,NaN,4.0,Volcanic activity,15,39
4,1902,Guatemala,NaN,10.0,Volcanic activity,8,37
5,1903,Canada,"Frank, Alberta",4.0,Mass movement (dry),10,11
7,1904,Bangladesh,Chittagong,11.0,Storm,83,20


In [11]:
# Pasar la columna 'Year' a tipo int
df['Year'] = df['Year'].astype(int)
df.head()

,Year,Country,Location,Month,Disaster Type,Precipitation (MM),Temperature
2,1902,Guatemala,"Quezaltenango, San Marcos",4.0,Earthquake,32,15
3,1902,Guatemala,NaN,4.0,Volcanic activity,15,39
4,1902,Guatemala,NaN,10.0,Volcanic activity,8,37
5,1903,Canada,"Frank, Alberta",4.0,Mass movement (dry),10,11
7,1904,Bangladesh,Chittagong,11.0,Storm,83,20


In [12]:
# Dentro del df columna 'Location' separar las filas que contengan ';' en dos filas
df = df.assign(Location=df['Location'].str.split(';')).explode('Location')
df.head()

,Year,Country,Location,Month,Disaster Type,Precipitation (MM),Temperature
2,1902,Guatemala,"Quezaltenango, San Marcos",4.0,Earthquake,32,15
3,1902,Guatemala,NaN,4.0,Volcanic activity,15,39
4,1902,Guatemala,NaN,10.0,Volcanic activity,8,37
5,1903,Canada,"Frank, Alberta",4.0,Mass movement (dry),10,11
7,1904,Bangladesh,Chittagong,11.0,Storm,83,20


In [13]:
# Buscar en 'location' El Paso (Texas)
aux = df[df['Location'] == 'El Paso (Texas)']
aux.head()

,Year,Country,Location,Month,Disaster Type,Precipitation (MM),Temperature
16099,2021,United States of America (the),El Paso (Texas),8.0,Flood,87,23


In [20]:
# Recorrer todos los años desde 1902 hasta 2021 y entre los meses 1 y 12, si un año no tiene un mes, añadirlo con el valor del mes correspondiente
for year in range(1900, 2021):
    for month in range(1, 12):
        # print(year, month)
        if not df[(df['Year'] == year) & (df['Month'] == month)].any().any():
            # print("entro")
            # random de estados de estados unidos
            states = ['Alabama', 'Alaska', 'Arizona', 'Arkansas', 'California', 'Colorado', 'Connecticut', 'Delaware', 'Florida', 
                    'Georgia', 'Hawaii', 'Idaho', 'Illinois', 'Indiana', 'Iowa', 'Kansas', 'Kentucky', 'Louisiana', 
                    'Maine', 'Maryland', 'Massachusetts', 'Michigan', 'Minnesota', 'Mississippi', 'Missouri', 'Montana',
                    'Nebraska', 'Nevada', 'New Hampshire', 'New Jersey', 'New Mexico', 'New York', 'North Carolina', 
                    'North Dakota', 'Ohio', 'Oklahoma', 'Oregon', 'Pennsylvania', 'Rhode Island', 'South Carolina', 
                    'South Dakota', 'Tennessee', 'Texas', 'Utah', 'Vermont', 'Virginia', 'Washington', 'West Virginia', 
                    'Wisconsin', 'Wyoming']
            
            for estado in states:
                # Temperatura random entre -10 y 40
                temp = np.random.randint(-10, 40)
                # Precipitacion random con distribucion normal
                precip = np.random.normal(50, 20)

                # crear una fila con los valores del año y el mes y dejar el resto de valores a como null
                new_row = {'Year': year, 'Month': month, 'Country': 'United States of America (the)', 'Location': estado, 'Disaster Type': "No", 'Precipitation (MM)': precip, 'Temperature': temp}
                # añadir la fila al df
                df = df.append(new_row, ignore_index=True)

# ocultar warnings
pd.options.mode.chained_assignment = None
# Mostrar df
df.head()


,Year,Country,Location,Month,Disaster_Type,Precipitation_(MM),Temperature,Disaster Probability (or prediction)
0,1902,Guatemala,"Quezaltenango, San Marcos",4.0,Earthquake,32.0,15,0
1,1902,Guatemala,NaN,4.0,Volcanic activity,15.0,39,0
2,1902,Guatemala,NaN,10.0,Volcanic activity,8.0,37,0
3,1903,Canada,"Frank, Alberta",4.0,Mass movement (dry),10.0,11,0
4,1904,Bangladesh,Chittagong,11.0,Storm,83.0,20,0


In [22]:
# Agregar la columna 'Disaster Probability (or prediction)' al df
df['Disaster Probability (or prediction)'] = 0
# En el df cuando df['Disaster Type'] es distinto de 'No' y NaN poner 1 en la columna 'Disaster Probability (or prediction)'
df.loc[(df['Disaster_Type'] != "No") & (df['Disaster_Type'].notnull()), 'Disaster_Probability_(or_prediction)'] = 1

# Mostrar df
df.head()

,Year,Country,Location,Month,Disaster_Type,Precipitation_(MM),Temperature,Disaster Probability (or prediction),Disaster_Probability_(or_prediction)
0,1902,Guatemala,"Quezaltenango, San Marcos",4.0,Earthquake,32.0,15,0,1.0
1,1902,Guatemala,NaN,4.0,Volcanic activity,15.0,39,0,1.0
2,1902,Guatemala,NaN,10.0,Volcanic activity,8.0,37,0,1.0
3,1903,Canada,"Frank, Alberta",4.0,Mass movement (dry),10.0,11,0,1.0
4,1904,Bangladesh,Chittagong,11.0,Storm,83.0,20,0,1.0


In [23]:
# Colum names replace with underscore
df.columns = df.columns.str.replace(' ', '_')

# guardar df en un csv
df.to_csv('../Data/extendido_1900_2021_DISASTERS.xlsx - emdat data.csv', index=False)

In [26]:
aux = df[(df['Year'] == 1902) & (df['Disaster_Type'] == "No")]
# primeros 10 valores
aux.head(10)


,Year,Country,Location,Month,Disaster_Type,Precipitation_(MM),Temperature,Disaster_Probability_(or_prediction),Disaster_Probability_(or_prediction)
17391,1902,United States of America (the),Alabama,1.0,No,77.711213,-7,0,NaN
17392,1902,United States of America (the),Alaska,1.0,No,41.977267,22,0,NaN
17393,1902,United States of America (the),Arizona,1.0,No,32.080843,-1,0,NaN
17394,1902,United States of America (the),Arkansas,1.0,No,52.550798,37,0,NaN
17395,1902,United States of America (the),California,1.0,No,60.710255,37,0,NaN
17396,1902,United States of America (the),Colorado,1.0,No,83.845178,-2,0,NaN
17397,1902,United States of America (the),Connecticut,1.0,No,38.463401,34,0,NaN
17398,1902,United States of America (the),Delaware,1.0,No,19.798506,-9,0,NaN
17399,1902,United States of America (the),Florida,1.0,No,28.804960,-4,0,NaN
17400,1902,United States of America (the),Georgia,1.0,No,41.900840,-2,0,NaN


In [40]:
# Separar df en data de entrenamiento y data de test de forma aleatoria
df_train = df.sample(frac=0.8, random_state=0)
df_test = df.drop(df_train.index)

# del df_test eliminar la columna 'Disaster Probability (or prediction)'
df_test = df_test.drop(columns=['Disaster Probability (or prediction)'])

# Mostrar df_train
df_train.head()

# Mostrar df_test
df_test.head()


,Year,Country,Location,Month,Disaster Type,Precipitation (MM),Temperature
10,1906,Belgium,NaN,4.0,Flood,74.0,-2
13,1908,Canada,"Notre Dame de La Salette, Quebec",4.0,Mass movement (dry),89.0,16
19,1910,Costa Rica,Cartago,4.0,Earthquake,74.0,26
28,1916,India,"Cuddalore, Pondicherry",11.0,Storm,90.0,-2
40,1921,Anguilla,NaN,1.0,Epidemic,72.0,7
